### **🔹 Project Introduction: Fine-Tuning IBM Granite-8B for Dyslexia Accessibility**  

This project aims to fine-tune the **IBM Granite-8B** model to enhance **text simplification** for individuals with dyslexia. The objective is to enable the model to rewrite complex sentences into clearer and more readable versions while preserving their original meaning. Using **Supervised Fine-Tuning (SFT) with PEFT (LoRA)**, the model learns to generate simplified text effectively.  

The fine-tuned model is deployed on **Hugging Face** and can be integrated into real-time accessibility tools. This innovation will help improve reading comprehension and accessibility for people with dyslexia.  

✅ **Key Features:**  
- **Text Simplification**: Makes sentences easier to read and understand.  
- **Efficient Fine-Tuning**: Uses **LoRA (Low-Rank Adaptation)** for optimized model adaptation.  
- **Deployable via API**: Accessible through **Hugging Face** for integration into applications.  

🚀 **Goal**: To build an AI-powered tool that enhances reading accessibility for dyslexic individuals.  

In [1]:
!pip install datasets transformers accelerate torch peft

In [2]:
!pip install -U bitsandbytes

In [3]:
import pandas as pd
import torch
from transformers import BitsAndBytesConfig, pipeline

Chargement et Initialisation du Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "ibm-granite/granite-3.0-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

Chargement et Configuration du Modèle

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=3,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)

model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
model.print_trainable_parameters()

trainable params: 5,222,400 || all params: 8,176,070,656 || trainable%: 0.0639


In [5]:
sentence = "The boy is playing soccer with his friends in the park."
prompt = f"""
  You are an AI assistant designed to simplify text for individuals with dyslexia.
  Your goal is to rewrite sentences in a clearer and more readable way while keeping their original meaning.

  Rewrite the following sentence in a simpler way:
  {"sentence"}
"""

In [ ]:
chat = [
    { "role": prompt},
    { "user": sentence}
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
input_tokens = tokenizer(chat, return_tensors="pt")

# Move tensors to the device
input_ids = input_tokens.input_ids.to(device)  # Move input_ids to the device
attention_mask = input_tokens.attention_mask.to(device)  # Move attention_mask to the device

# Generate output tokens
output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=100)

In [ ]:
# decode output tokens into text
output = tokenizer.batch_decode(output, skip_special_tokens=True)
# print output
print(output)

["assistantI'm here to help! What's your question?"]


In [ ]:
from datasets import load_dataset

ds = load_dataset("facebook/asset", "simplification")

README.md:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/885k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/170k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/359 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    validation: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 359
    })
})

In [ ]:
train_data = ds['validation']
test_data = ds['test']

In [ ]:
train_data

Dataset({
    features: ['original', 'simplifications'],
    num_rows: 2000
})

In [ ]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

**Vérification des dimensions**

In [ ]:
train_data.shape, test_data.shape

((2000, 2), (359, 2))

In [ ]:
for index, row in train_data.iterrows():
  i = index
  train_data.loc[index, 'prompt'] = f"""
    You are an AI assistant designed to simplify text for individuals with dyslexia.
    Your goal is to rewrite sentences in a clearer and more readable way while keeping their original meaning.

    Rewrite the following sentence in a simpler way:
    {row["original"]}
  """
  train_data.loc[index, 'completion'] = row["simplifications"][0]

In [ ]:
for index, row in test_data.iterrows():
  i = index
  test_data.loc[index, 'prompt'] = f"""
    You are an AI assistant designed to simplify text for individuals with dyslexia.
    Your goal is to rewrite sentences in a clearer and more readable way while keeping their original meaning.

    Rewrite the following sentence in a simpler way:
    {row["original"]} # Access using attribute name instead of string key
  """
  test_data.loc[index, 'completion'] = row["simplifications"][0]

In [ ]:
X_train_data = train_data['prompt'].values
y_train_data = train_data['completion'].values

In [ ]:
X_test_data = test_data['prompt'].values
y_test_data = test_data['completion'].values

In [ ]:
X_test_data.shape, y_test_data.shape

((359,), (359,))

In [ ]:
# Tokenisation des entrées
X_train_tokens = tokenizer(X_train_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

# Tokenisation des sortie
y_train_tokens = tokenizer(y_train_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

In [ ]:
# Tokenisation des entrées
X_test_tokens = tokenizer(X_test_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
# Tokenisation des sortie
y_test_tokens = tokenizer(y_test_data.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

In [ ]:
from datasets import Dataset

tokenized_train_dataset = Dataset.from_dict({
    "input_ids": X_train_tokens["input_ids"],
    "attention_mask": X_train_tokens["attention_mask"],
    "labels": y_train_tokens["input_ids"]
})

tokenized_test_dataset = Dataset.from_dict({
    "input_ids": X_test_tokens["input_ids"],
    "attention_mask": X_test_tokens["attention_mask"],
    "labels": y_test_tokens["input_ids"]
})

In [4]:
import os
from accelerate import Accelerator

# Setup environment variables for GPU efficiency
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "dryrun"

# Initialize Accelerator
accelerator = Accelerator(mixed_precision="fp16")

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Sat Feb 22 11:10:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             52W /  400W |    6121MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!kill -9 <pid>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `kill -9 <pid>'


In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
!pip install rouge_score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=18478866437a76420d6cfed7bbef25e4fe34924ba5b53cf02f17c4b7b0c968f2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00


In [ ]:
from evaluate import load

# Charger les métriques d'évaluation
rouge = load("rouge")
bleu = load("sacrebleu")

# Définir la fonction pour évaluer le modèle
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Convert predictions and labels to lists of lists
    preds = preds.tolist()  # Convert preds NumPy array to a list of lists
    labels = labels.tolist() # Convert labels NumPy array to a list of lists

    # Convertir les tokens en texte
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE Score
    rouge_scores = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # BLEU Score
    bleu_score = bleu.compute(predictions=[decoded_preds], references=[[decoded_labels]])

    # Afficher les résultats
    result = {
        "rouge1": rouge_scores["rouge1"].mid.fmeasure,
        "rouge2": rouge_scores["rouge2"].mid.fmeasure,
        "rougeL": rouge_scores["rougeL"].mid.fmeasure,
        "bleu": bleu_score["score"]
    }

    return result

### **Définition des Paramètres d'Entraînement :**

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./granite_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    gradient_accumulation_steps=16,
    fp16=True,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    optim="adamw_torch",
    report_to="none",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### **Initialisation du Trainer :**

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.1 MB/s eta 0:00:00


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-41-e7fb46f110fc>:4: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/359 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/359 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/359 [00:00<?, ? examples/s]

### **Lancement de l'Entraînement :**

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.945700,0.729439
2,0.650600,0.740707
3,0.587200,0.758666


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=375, training_loss=0.6950705057779948, metrics={'train_runtime': 3675.5242, 'train_samples_per_second': 1.632, 'train_steps_per_second': 0.102, 'total_flos': 4.3350641934336e+16, 'train_loss': 0.6950705057779948})

In [ ]:
model.save_pretrained("./granite_finetuned_model_v2")
tokenizer.save_pretrained("./granite_finetuned_model_v2")

('./granite_finetuned_model_v2/tokenizer_config.json',
 './granite_finetuned_model_v2/special_tokens_map.json',
 './granite_finetuned_model_v2/vocab.json',
 './granite_finetuned_model_v2/merges.txt',
 './granite_finetuned_model_v2/added_tokens.json',
 './granite_finetuned_model_v2/tokenizer.json')

In [ ]:
import json

config = model.config

# Convert the GraniteConfig object to a dictionary
config_dict = config.to_dict()

with open("./granite_finetuned_model_v2/config.json", "w") as f:
    json.dump(config_dict, f, indent=4)

In [ ]:
import torch
from safetensors.torch import load_file, save_file

# Charger le modèle sauvegardé
model_weights = load_file("/content/granite_finetuned_model_v2/adapter_model.safetensors")

# Sauvegarder au format PyTorch
torch.save(model_weights, "/content/granite_finetuned_model_v2/pytorch_model.bin")

print("Model converted to pytorch_model.bin")

In [ ]:
!zip -r ./granite_finetuned_model_v3.zip ./granite_finetuned_model_v2/

  adding: granite_finetuned_model_v2/ (stored 0%)
  adding: granite_finetuned_model_v2/added_tokens.json (deflated 34%)
  adding: granite_finetuned_model_v2/tokenizer_config.json (deflated 86%)
  adding: granite_finetuned_model_v2/special_tokens_map.json (deflated 73%)
  adding: granite_finetuned_model_v2/tokenizer.json (deflated 81%)
  adding: granite_finetuned_model_v2/adapter_model.safetensors (deflated 9%)
  adding: granite_finetuned_model_v2/README.md (deflated 66%)
  adding: granite_finetuned_model_v2/adapter_config.json (deflated 55%)
  adding: granite_finetuned_model_v2/config.json (deflated 64%)
  adding: granite_finetuned_model_v2/vocab.json (deflated 57%)
  adding: granite_finetuned_model_v2/merges.txt (deflated 51%)
  adding: granite_finetuned_model_v2/pytorch_model.bin (deflated 9%)


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel

def load_model():
    # Charger le modèle de base IBM Granite
    base_model = AutoModelForCausalLM.from_pretrained(
        "ibm-granite/granite-3.0-8b-instruct",
        torch_dtype=torch.float32,
        device_map="auto"
    )

    # Charger le modèle fine-tuné avec LoRA
    model = PeftModel.from_pretrained(base_model, "IAyamina/IBM_hackathonIA_granite_finetuned_dyslexia")

    # Charger le tokenizer
    tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.0-8b-instruct")

    return model, tokenizer

# Charger le modèle et tokenizer
model, tokenizer = load_model()

def simplify_text(text):
    prompt = f"""
    You are an AI assistant designed to simplify text for individuals with dyslexia.
    Your sould rewrite sentences in a very simple and clearer and more readable way while keeping their meaning.

    Rewrite the following sentence in a simpler way:
    {text}
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Envoyer sur GPU si disponible
    outputs = model.generate(**inputs, max_new_tokens=500)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
# Tester la simplification
text =
"The proliferation of computational methodologies in contemporary artificial intelligence research has significantly augmented the capacity of neural networks to discern intricate patterns within multidimensional datasets. This paradigm shift has engendered an unprecedented acceleration in the automation of cognitive processes, fostering advancements in fields as disparate as biomedical engineering and quantum computing."
simplified_text = simplify_text(text)

print("Simplified Text:", simplified_text)

Simplified Text:     
    You are an AI assistant designed to simplify text for individuals with dyslexia.
    Your sould rewrite sentences in a very simple and clearer and more readable way while keeping their meaning.

    Rewrite the following sentence in a simpler way:
    The proliferation of computational methodologies in contemporary artificial intelligence research has significantly augmented the capacity of neural networks to discern intricate patterns within multidimensional datasets. This paradigm shift has engendered an unprecedented acceleration in the automation of cognitive processes, fostering advancements in fields as disparate as biomedical engineering and quantum computing.
     A simpler alternative could be:
    The proliferation of computational methodologies in contemporary artificial intelligence research has significantly augmented the capacity of neural networks to discern intricate patterns within multidimensional datasets. This has led to an unprecedented ac

# **Streamlit Interface:**

In [1]:
!pip install streamlit torch transformers peft

In [1]:
%%writefile app.py
import streamlit as st
import torch
import re  # ✅ FIX: Importing regex module
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 🏗️ Load the fine-tuned model and tokenizer
@st.cache_resource
def load_model():
    base_model = AutoModelForCausalLM.from_pretrained(
        "ibm-granite/granite-3.0-8b-instruct",
        torch_dtype=torch.bfloat16,  # Optimizing memory usage
        device_map="auto"
    )

    # Load fine-tuned weights
    model = PeftModel.from_pretrained(base_model, "IAyamina/IBM_hackathonIA_granite_finetuned_dyslexia")

    tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.0-8b-instruct")

    return model, tokenizer

# Load the model and tokenizer
model, tokenizer = load_model()

# ✨ Function to simplify text
def simplify_text(text):
    prompt = f"Rewrite the following sentence in a simpler way: {text}"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move to GPU if available
    outputs = model.generate(**inputs, max_new_tokens=100)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # ✅ Extract only the part after "Answer:"
    match = re.search(r"Answer:\s*(.*)", generated_text, re.IGNORECASE)
    if match:
        return match.group(1).strip()  # ✅ Return only the simplified sentence
    else:
        return generated_text.strip()  # ✅ Return the cleaned output

# 🎨 Streamlit Interface
st.title("🧠 AI-Powered Dyslexia Text Simplifier")
st.write("Enter a complex text, and the model will simplify it for better readability!")

# User input text box
user_input = st.text_area("Enter your complex text:", "The proliferation of computational methodologies...")

if st.button("Simplify Text"):
    with st.spinner("Processing... 🔄"):
        simplified_text = simplify_text(user_input)
    st.success("✅ Simplification Complete!")
    st.write("**Simplified Text:**")
    st.write(simplified_text)


Overwriting app.py


In [2]:
!pip install pyngrok

In [3]:
!ngrok authtoken <your_authtoken>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:

from pyngrok import ngrok

# Démarrer un tunnel HTTP sur le port 8501
public_url = ngrok.connect(8501, "http")
print(f"Votre application Streamlit est accessible à cette URL : {public_url}")
# Lancer Streamlit
!streamlit run app.py --server.port 8501 >/dev/null 2>&1 &

Votre application Streamlit est accessible à cette URL : NgrokTunnel: "https://6bf4-35-192-18-203.ngrok-free.app" -> "http://localhost:8501"
